In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# from openbabel import pybel
#from rdkit.Chem import PandasTools
#from rdkit.Chem import SDMolSupplier
import numpy as np
import os
import subprocess
from subprocess import Popen, PIPE
import pandas as pd
from time import sleep

In [125]:
#prueba:
# tar='esr1_ago'

In [3]:
# icm_or_filename = tar.split('_')[0]+'_AF_answers1_unique.sdf'
# icm_command='/home/ccgroup/icm-browser-pro-3.9-2d/icmbrowserpro64'
cwd = "/media/emilia/HDD"
inputs_dir=cwd+'/Documents/new_excels/'
outputs_dir=cwd+'/Documents/new_excels/'
outputs_dir=cwd+'/PRC_outputs/'

In [127]:
#out_consensus = pd.read_excel(inputs_dir+tar+'_consensus_5p_ICM_conf.xlsx')
#out_consensus = pd.read_csv(outputs_dir_tar+tar+'_consensus_added.csv')
# out_consensus = pd.read_excel(outputs_dir_tar+tar+'_consensus_5p.xlsx')

In [131]:
# TOTAL MOLS:
N = len(out_consensus)
N

21991

In [78]:
#Hacer rename
for comb in out_consensus.columns:
    if (comb.split('_')[0] == 'ecr'):
        out_consensus = out_consensus.rename(columns={comb : '_'.join(['ecr','_'.join(sorted(comb.split('_')[1:]))])})

In [143]:
dict_ranks = {'rank_ICM' : 'rank_icm' , 'rank_rDock' : 'rank_rd' , 'rank_dock' : 'rank_dock6' ,'rank_AD' : 'rank_ad' , 'rank_PL' : 'rank_pl'}

In [144]:
out_consensus = out_consensus.rename(columns=dict_ranks)

In [153]:
# out_consensus.columns

Index(['Unnamed: 0', 'mol', 'Type', 'rank_icm', 'rank_rd', 'rank_ad',
       'rank_pl', 'rank_dock6', 'rank_vina', 'exp_icm', 'exp_rd', 'exp_ad',
       'exp_pl', 'exp_dock6', 'exp_vina', 'RMSD_AD_ICM', 'RMSD_AD_RDOCK',
       'RMSD_PL_AD', 'RMSD_PL_ICM', 'RMSD_PL_RDOCK', 'RMSD_ICM_RDOCK',
       'RMSD_ICM_VINA', 'RMSD_AD_VINA', 'RMSD_VINA_RDOCK', 'RMSD_PL_VINA',
       'RMSD_DOCK_PL', 'RMSD_DOCK_AD', 'RMSD_DOCK_RDOCK', 'RMSD_DOCK_ICM',
       'RMSD_DOCK_VINA', 'Pose Cons Level Max', 'cons progs', 'exp_icm_rd',
       'exp_ad_icm', 'exp_dock6_icm', 'exp_icm_pl', 'exp_ad_rd',
       'exp_dock6_rd', 'exp_pl_rd', 'exp_ad_dock6', 'exp_ad_pl',
       'exp_dock6_pl', 'exp_ad_icm_rd', 'exp_dock6_icm_rd', 'exp_icm_pl_rd',
       'exp_ad_dock6_icm', 'exp_ad_icm_pl', 'exp_dock6_icm_pl',
       'exp_ad_dock6_rd', 'exp_ad_pl_rd', 'exp_dock6_pl_rd', 'exp_ad_dock6_pl',
       'exp_ad_dock6_icm_rd', 'exp_ad_icm_pl_rd', 'exp_dock6_icm_pl_rd',
       'exp_ad_dock6_icm_pl', 'exp_ad_dock6_pl_rd', 'exp_ad

In [92]:
thr2=0
thr3=0.02
thr4=0.05
thr5=0.1
thr_ecr = 0.05 #Estos hay que comentarlos y definirlos en la notebook anterior. 

prc_passing_mols = np.zeros(len(out_consensus))

for index, row in out_consensus.iterrows():
    coincidents = eval(row['Max coincident progs'])
    coincidents = [sorted(arr) for arr in coincidents]
    for arr in coincidents:
        if('DOCK' in arr):
            arr[arr.index('DOCK')] = 'DOCK6'
        if('RDOCK' in arr):
            arr[arr.index('RDOCK')] = 'RD'
#     rmsd_values = row[['RMSD_PL_ICM', 'RMSD_PL_RDOCK', 'RMSD_ICM_RDOCK', 'RMSD_AD_ICM', 'RMSD_AD_RDOCK', 'RMSD_PL_AD', 'RMSD_DOCK_PL', 'RMSD_DOCK_AD',
#        'RMSD_DOCK_RDOCK', 'RMSD_DOCK_ICM']]
# #     cons_level=1
#     if( any(rmsd_values < thr)):
# #         cons_level += 1
#          coincident_poses = [ rmsd_values.index[i] for i in range(len(rmsd_values)) if rmsd_values[i] < thr]
#          cons_prog = [rmsd_progs_dict[x] for x in coincident_poses]
# #          if( len(coincident_poses) > 2):
# #             if( all(rmsd_values < thr) ):
# # #                 cons_level += 3
# #                 cons_prog = ['ad_rd_icm_pl_dock6']  
    
    if(row['Max pose cons level'] == 2):
        for comb in coincidents:
            if('AD' in comb):
                ecr_name = 'rank_'+comb[1].lower()
            else:
                ecr_name = 'ecr_'+comb[0].lower()+'_'+comb[1].lower()
                
            if( row[ecr_name] < int(thr2*N )):
                prc_passing_mols[index] = 1
                break
        
    elif(row['Max pose cons level'] == 3):
        for comb in coincidents:
            if('AD' in comb):
                ecr_name='ecr_'+comb[1].lower()+'_'+comb[2].lower()
            else:
                ecr_name = 'ecr_'+comb[0].lower()+'_'+comb[1].lower()+'_'+comb[2].lower()
            if( row[ecr_name] < int(thr3*N )):
                prc_passing_mols[index] = 1
                break
    
    elif(row['Max pose cons level'] == 4):
        for comb in coincidents:
            if('AD' in comb):
                ecr_name='ecr_'+comb[1].lower()+'_'+comb[2].lower()+'_'+comb[3].lower()
            else:
                ecr_name = 'ecr_'+comb[0].lower()+'_'+comb[1].lower()+'_'+comb[2].lower()+'_'+comb[3].lower()
            if( row[ecr_name] < int(thr4*N )):
                prc_passing_mols[index] = 1
                break
                
    elif(row['Max pose cons level'] == 5):            
        #coincidents=combinations[-1][4:].split('_')
        #ecr_name = 'ecr_'+coincidents[0]+'_'+coincidents[1]+'_'+coincidents[2]+'_'+coincidents[3]+'_'+coincidents[4]
        ecr_name = 'ecr_dock6_icm_pl_rd'
        if( row[ecr_name] < int(thr5*N )):
                prc_passing_mols[index] = 1

out_consensus['Final selection 5p'] = prc_passing_mols

In [155]:
#option D
num_ligs_d = len(out_consensus.loc[(out_consensus['Final selection 5p'] == 1 ) & (out_consensus['Type'] == 'L')])
num_tot_d = len(out_consensus.loc[(out_consensus['Final selection 5p'] == 1 )])
num_tot_ligands = len(out_consensus.loc[(out_consensus['Type'] == 'L')])

# print(num_ligs_d, num_tot_d)
# print("Hit Rate Option D" , num_ligs_d/num_tot_d)

150 392
Hit Rate Option D 0.3826530612244898


In [156]:
#PRC
thr_ecr_local=0.01
num_ligs_prc = len(out_consensus.loc[(out_consensus['Final selection 5p'] == 1 ) & (out_consensus['Type'] == 'L') & (out_consensus['ecr_dock6_icm_pl_rd'] < int(thr_ecr*N))])
num_tot_prc = len(out_consensus.loc[(out_consensus['Final selection 5p'] == 1 ) & (out_consensus['ecr_dock6_icm_pl_rd'] < int(thr_ecr*N))])
# print(num_ligs_prc, num_tot_prc)
# print("Hit Rate" , num_ligs_prc/num_tot_prc)

127 210
Hit Rate 0.6047619047619047


In [ ]:
#ECR 1%
#order out_consensus by ecr_ad_dock6_pl_rd_vina ascending

out_consensus = out_consensus.sort_values(by=['ecr_ad_dock6_pl_rd_vina'], axis=0)
num_ligs_ecr = len(out_consensus.loc[(out_consensus['Type'] == 'L') & (out_consensus['ecr_ad_dock6_icm_pl_rd'] < int(thr_ecr_local*N))])  
num_tot_ecr = len(out_consensus.loc[(out_consensus['ecr_ad_dock6_icm_pl_rd'] <= thr_ecr_local*N)])
if (num_tot_ecr == 0):
    print("No ligands with ECR <= 2%")
else:
    ecr1 = (num_ligs_ecr/ num_tot_ecr)